# Ocular Disease Recognition

This notebook is focus on modeling using TPOT and PyTorch as feature extractor.

In [1]:
import pandas as pd
import numpy as np
import os
import ast

## 1. Data preparationimport pandas as pd
import matplotlib.pyplot as plt
import os
import ast

In [2]:
PATH_DATA = "/kaggle/input/ocular-disease-recognition-odir5k"

In [3]:
df_full = pd.read_csv(os.path.join(PATH_DATA, "full_df.csv"))
print("Shape:", df_full.shape)
df_full.head()

Shape: (6392, 19)


,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


### 1.1. Target variable

Revome noise in target variable.

In [4]:
# Convert list-form value into list and get the first value
df_full["labels"] = df_full["labels"].map(lambda x: ast.literal_eval(x)[0])

Turn string value into numeric

In [5]:
def get_label_encode(x):
    if x == "N":
        return 0
    elif x == "D":
        return 1
    elif x == "G":
        return 2
    elif x == "C":
        return 3
    elif x == "A":
        return 4
    elif x == "H":
        return 5
    elif x == "M":
        return 6
    else:
        return 7

In [6]:
df_full["labels"] = df_full["labels"].apply(get_label_encode)
df_full.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,1,"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,1,"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,1,"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


### 1.2. Image path

Get full path image

In [7]:
df_full["filename"] = df_full["filename"].map(lambda x: os.path.join(PATH_DATA, "ODIR-5K", "ODIR-5K", "Training Images", x))
df_full.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",/kaggle/input/ocular-disease-recognition-odir5...
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",/kaggle/input/ocular-disease-recognition-odir5...
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,1,"[0, 1, 0, 0, 0, 0, 0, 0]",/kaggle/input/ocular-disease-recognition-odir5...
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,1,"[0, 1, 0, 0, 0, 0, 0, 0]",/kaggle/input/ocular-disease-recognition-odir5...
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,1,"[0, 1, 0, 0, 0, 0, 0, 0]",/kaggle/input/ocular-disease-recognition-odir5...


### 1.3. Get only necessary features

In [8]:
df_train_full = df_full[["filename", "labels"]]
print("Shape:", df_train_full.shape)
df_train_full.head()

Shape: (6392, 2)


,filename,labels
0,/kaggle/input/ocular-disease-recognition-odir5...,0
1,/kaggle/input/ocular-disease-recognition-odir5...,0
2,/kaggle/input/ocular-disease-recognition-odir5...,1
3,/kaggle/input/ocular-disease-recognition-odir5...,1
4,/kaggle/input/ocular-disease-recognition-odir5...,1


## 2. Modeling

In [9]:
import torch
import torchvision
from torchvision.io import read_image
from torchvision import tv_tensors, models
from torchvision.transforms import v2 as T

from tpot import TPOTClassifier

In [10]:
print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")

Torch version: 2.1.2+cpu
Torchvision version: 0.16.2+cpu


### 2.1. Split data into training and validation

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
df_train, df_val = train_test_split(df_train_full.head(100), test_size = 0.2, random_state = 42)

print("df_train shape:", df_train.shape)
print("df_val shape:", df_val.shape)

df_train shape: (80, 2)
df_val shape: (20, 2)


### 2.2. Create dataset constructor

In [13]:
class OcularDataset(torch.utils.data.Dataset):
    def __init__(self, ds, transforms):
        self.transforms = transforms

        # Get list of image paths
        self.imgs = ds["filename"].values
        self.classes = ds["labels"].values

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        # Get image path
        img_path = self.imgs[idx]

        # Get target
        target = self.classes[idx]

        # Read image as RGB Tensor (3 channels)
        img = read_image(img_path)#.float() / 255.0

        # Wrap sample and targets into torchvision tv_tensors in order improve transform performance
        img = tv_tensors.Image(img)

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

In [14]:
def get_transform(train):
    transforms = []

    transforms.append(T.Resize((224, 224)))
    transforms.append(T.ToTensor())
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))

    return T.Compose(transforms)

In [15]:
dataset_train = OcularDataset(df_train, get_transform(train = True))
dataset_val = OcularDataset(df_val, get_transform(train = False))

data_loader_train = torch.utils.data.DataLoader(
    dataset = dataset_train,
    batch_size = 16,
    shuffle = True
)

data_loader_val = torch.utils.data.DataLoader(
    dataset = dataset_val,
    batch_size = 16,
    shuffle = False
)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


### 2.3. Training

#### 2.3.1. RestNet50

##### 2.3.1.1. Feature extraction

In [16]:
from torchvision.models import resnet50, ResNet50_Weights

def extract_features_batch_resnet50(batch_images):
    model = resnet50(weights = ResNet50_Weights.IMAGENET1K_V2)

    with torch.no_grad():
        features = model(batch_images)
        
    return features

**Training data**

In [17]:
# Extract features for the entire dataset
all_train_features = []
all_train_labels = []

# Iterate over the dataloader and extract features for each batch
for imgs, targets in data_loader_train:
    batch_features = extract_features_batch_resnet50(imgs)
    all_train_features.append(batch_features)
    all_train_labels.append(targets)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 98.6MB/s]


**Validation data**

In [18]:
# Extract features for the entire dataset
all_val_features = []
all_val_labels = []

# Iterate over the dataloader and extract features for each batch
for imgs, targets in data_loader_val:
    batch_features = extract_features_batch_resnet50(imgs)
    all_val_features.append(batch_features)
    all_val_labels.append(targets)

**Transforming into numpy array**

All features are being stored and transformed into numpy arrays, which are going to be the input for classification model.

In [19]:
x_train = np.concatenate(all_train_features, axis=0)
y_train = np.concatenate(all_train_labels, axis=0)

x_val = np.concatenate(all_val_features, axis=0)
y_val = np.concatenate(all_val_labels, axis=0)

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

print('x_val shape:', x_val.shape)
print('y_val shape:', y_val.shape)

x_train shape: (80, 1000)
y_train shape: (80,)
x_val shape: (20, 1000)
y_val shape: (20,)


##### 2.3.1.2. Classification

In [20]:
tpot = TPOTClassifier(generations = 2, population_size = 100, verbosity = 2, random_state = 42, n_jobs = -1, config_dict = "TPOT light")
tpot.fit(x_train, y_train)

Version 0.12.1 of tpot is outdated. Version 0.12.2 was released Friday February 23, 2024.


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklea


Generation 1 - Current best internal CV score: 0.35


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklea


Generation 2 - Current best internal CV score: 0.3875

Best pipeline: LogisticRegression(StandardScaler(input_matrix), C=20.0, dual=False, penalty=l2)


TPOTClassifier(config_dict='TPOT light', generations=2, n_jobs=-1,
               random_state=42, verbosity=2)

In [21]:
accuracy = tpot.score(x_val, y_val)
print(f"Accuracy: {accuracy}")

Accuracy: 0.3
